# Predicción de Ingresos — Resumen Ejecutivo

Objetivo: construir y evaluar un Random Forest que clasifique si una persona gana >50K/año, mostrando buenas prácticas en limpieza, EDA, transformaciones, modelo Random Forest y evaluación.

Datos: Adult Income Dataset (UCI). Fuente: https://archive.ics.uci.edu/dataset/2/adult

Notas rápidas:
- Contenido del proyecto, dependencias y versiones en `README.md` / `environment.yml`.
- Celdas costosas marcadas con "Nota: puede tardar".

## Carga de datos, librerías y primera inspección
Se carga el dataset con las librerias a usar en el notebook y se realiza una inspección inicial (info, head) para entender su estructura y tamaño.

In [1]:
# Limpieza y análisis de datos
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Ajuste de los datos
from sklearn.model_selection import train_test_split
from category_encoders import TargetEncoder
from sklearn.preprocessing import OneHotEncoder

# Creación del modelo
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

# Evaluación del modelo
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

from imblearn.pipeline import Pipeline

%run pandas-missing-extension.ipynb

df = pd.read_csv('https://archive.ics.uci.edu/static/public/20/data.csv', sep = ',')

df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48842 entries, 0 to 48841
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             48842 non-null  int64 
 1   workclass       47879 non-null  object
 2   fnlwgt          48842 non-null  int64 
 3   education       48842 non-null  object
 4   education-num   48842 non-null  int64 
 5   marital-status  48842 non-null  object
 6   occupation      47876 non-null  object
 7   relationship    48842 non-null  object
 8   race            48842 non-null  object
 9   sex             48842 non-null  object
 10  capital-gain    48842 non-null  int64 
 11  capital-loss    48842 non-null  int64 
 12  hours-per-week  48842 non-null  int64 
 13  native-country  48568 non-null  object
 14  income          48842 non-null  object
dtypes: int64(6), object(9)
memory usage: 5.6+ MB


,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


## Limpieza de Datos
### Valores Nulos

In [2]:
df.missing.missing_variable_summary()
cols_missing = ['workclass', 'occupation', 'native-country']

for col in cols_missing:
    print(f'Valores únicos de {col}:', df[col].unique())
    print('=' * 50)

Valores únicos de workclass: ['State-gov' 'Self-emp-not-inc' 'Private' 'Federal-gov' 'Local-gov' '?'
 'Self-emp-inc' 'Without-pay' 'Never-worked' nan]
Valores únicos de occupation: ['Adm-clerical' 'Exec-managerial' 'Handlers-cleaners' 'Prof-specialty'
 'Other-service' 'Sales' 'Craft-repair' 'Transport-moving'
 'Farming-fishing' 'Machine-op-inspct' 'Tech-support' '?'
 'Protective-serv' 'Armed-Forces' 'Priv-house-serv' nan]
Valores únicos de native-country: ['United-States' 'Cuba' 'Jamaica' 'India' '?' 'Mexico' 'South'
 'Puerto-Rico' 'Honduras' 'England' 'Canada' 'Germany' 'Iran'
 'Philippines' 'Italy' 'Poland' 'Columbia' 'Cambodia' 'Thailand' 'Ecuador'
 'Laos' 'Taiwan' 'Haiti' 'Portugal' 'Dominican-Republic' 'El-Salvador'
 'France' 'Guatemala' 'China' 'Japan' 'Yugoslavia' 'Peru'
 'Outlying-US(Guam-USVI-etc)' 'Scotland' 'Trinadad&Tobago' 'Greece'
 'Nicaragua' 'Vietnam' 'Hong' 'Ireland' 'Hungary' 'Holand-Netherlands' nan]


Al ver los valores únicos de cada columna, vemos que los valores faltantes están nombrados como "?" y "nan", entonces los reemplazamos por np.nan para tenerlos en una misma categoría.

In [3]:
df.replace('?', np.nan, inplace=True)
df.missing.missing_variable_summary()

,variable,n_missing,n_cases,pct_missing
0,age,0,48842,0.000000
1,workclass,2799,48842,5.730724
2,fnlwgt,0,48842,0.000000
3,education,0,48842,0.000000
4,education-num,0,48842,0.000000
5,marital-status,0,48842,0.000000
6,occupation,2809,48842,5.751198
7,relationship,0,48842,0.000000
8,race,0,48842,0.000000
9,sex,0,48842,0.000000


Vemos cómo los valores nulos en cada columna aumentaron en gran cantidad después de hacer el reemplazo. Ahora, lo que se hará será hacer otra imputación, pero esta vez convirtiendo los valores faltantes categóricos en una nueva categoría llamada `"Unknown"`, así no se pierde información y se puede seguir trabajando con esos datos.

In [4]:
df['workclass'] = df['workclass'].fillna('Unknown')
df['occupation'] = df['occupation'].fillna('Unknown')
df['native-country'] = df['native-country'].fillna('Unknown')
df.missing.missing_variable_summary()

,variable,n_missing,n_cases,pct_missing
0,age,0,48842,0.0
1,workclass,0,48842,0.0
2,fnlwgt,0,48842,0.0
3,education,0,48842,0.0
4,education-num,0,48842,0.0
5,marital-status,0,48842,0.0
6,occupation,0,48842,0.0
7,relationship,0,48842,0.0
8,race,0,48842,0.0
9,sex,0,48842,0.0


### Valores Duplicados

In [5]:
num_duplicados = df.duplicated().sum()
print(f"Número de filas duplicadas: {num_duplicados}")

Número de filas duplicadas: 29


Al ver que no son tantos datos duplicados, procedemos a eliminarlos ya que no afectará mucho el dataset y puede ayudar a mejorar el rendimiento del modelo.

In [6]:
df = df.drop_duplicates()
print(f"Número de filas duplicadas: {df.duplicated().sum()}")

df_processing = df.copy()

Número de filas duplicadas: 0


Finalmente, copiamos el dataframe limpio a una nueva variable para seguir trabajando con ella y no perder el dataframe original.